In [ ]:
#default_exp event

# The events API

> Helpers for getting GitHub API events

In [ ]:
#export
from fastcore.utils import *
from fastcore.foundation import *
from ghapi.core import *
from ghapi.page import *

import time
from itertools import islice

In [ ]:
api = GhApi()

In [ ]:
# TODO only new events

In [ ]:
#export
def _list_events(username=None, org=None, owner=None, repo=None):
    if (username or org or owner) and \
        not (bool(username) ^ bool(org) ^ bool(owner)): raise Exception('Can not pass more than one of username, org, and owner')
    if (owner and not repo) or (repo and not owner): 'Must pass both repo and owner, if passing either'
    if owner: return api.activity.list_public_events_for_repo_network,{'owner':owner,'repo':repo}
    if org: return api.activity.list_public_org_events,{'org':org}
    if username: return api.activity.list_public_events_for_user,{'username':username}
    return api.activity.list_public_events,{}

In [ ]:
#export
@patch
def list_events(self:GhApi, username=None, org=None, owner=None, repo=None, per_page=30, page=1):
    "Fetch public events for repo network, org, user, or all"
    oper,kwargs = _list_events(username=username, org=org, owner=owner, repo=repo)
    return oper(per_page=per_page, page=page, **kwargs)

In [ ]:
#export
@patch
def list_events_parallel(self:GhApi, username=None, org=None, owner=None, repo=None, per_page=30, n_pages=8):
    "Fetch as many events from `list_events` in parallel as available"
    oper,kwargs = _list_events(username=username, org=org, owner=owner, repo=repo)
    return pages(oper, n_pages, per_page=per_page, **kwargs).concat()

In [ ]:
ev = api.list_events_parallel(org='fastai')
len(ev),ev[0].org.login

(240, 'fastai')

In [ ]:
#export
def fetch_events(types=None):
    "Generate an infinite stream of events optionally filtered to `types`"
    if types: types=setify(types)
    seen = set()
    while True:
        evts = [o for o in api.list_events_parallel(n_pages=2) if o.id not in seen]
        print('\n***', len(evts))
        for o in evts:
            seen.add(o.id)
            if not types or o.type in types: yield o
        time.sleep(0.6)

In [ ]:
for o in islice(fetch_events('PullRequestEvent'), 100): print(o.id, end=' ')


*** 60
14505512015 14505511995 14505511993 14505511981 14505511975 14505511976 14505511971 14505511970 14505511968 14505511963 14505511921 14505511920 14505511889 
*** 28
14505512077 14505512069 14505512063 14505512061 14505512053 14505512051 14505512048 14505512040 14505512034 
*** 38
14505512248 14505512218 14505512215 14505512210 14505512198 14505512092 
*** 41
14505512340 14505512326 14505512325 14505512314 14505512313 14505512315 14505512311 14505512304 14505512307 14505512289 14505512277 14505512271 14505512261 14505512257 
*** 43
14505512456 14505512454 14505512450 14505512445 14505512429 14505512427 14505512408 14505512403 14505512395 14505512380 14505512366 14505512362 14505512355 14505512353 
*** 32
14505512515 14505512505 14505512504 14505512495 14505512486 14505512477 14505512458 
*** 38
14505512598 14505512600 14505512582 14505512566 14505512558 14505512549 14505512538 14505512530 14505512523 
*** 60
14505512763 14505512761 14505512740 14505512727 14505512723 14505512720 

In [ ]:
next(fetch_events('PullRequestEvent'))

In [ ]:
evts = GhEvents(types=('IssuesEvent','ReleaseEvent','PullRequestEvent'), bots=True)

In [ ]:
for ev in evts: ...

In [ ]:
for b in islice(evts,10): ...

## Export -

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_core.ipynb.
Converted 01_actions.ipynb.
Converted 02_auth.ipynb.
Converted 10_cli.ipynb.
Converted 50_fullapi.ipynb.
Converted 90_build_lib.ipynb.
Converted Untitled.ipynb.
Converted index.ipynb.
Converted tutorial_actions.ipynb.
